In [1]:
import together

In [2]:
together.api_key = "d62935f008f7b7799302e1c08e3d1d4ef3328b2183b629319a1d1a01505b4afa"

In [3]:

endpoint = 'https://api.together.xyz/inference'


def send_prompt(prompt):
    response = together.Complete.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        prompt=f"[INST] {prompt} [/INST]",
        max_tokens=10,
        temperature=0.7,
        stop=["\n"],
    )
    return response['output']['choices'][0]['text']


In [10]:
# let's create a retry wrapper for our function
# we need to have exponential backoff
import time

def retry_wrapper(func):
    def wrapper(*args, **kwargs):
        for i in range(3):
            try:
                return func(*args, **kwargs)
            except Exception as e:
                print(e)
                time.sleep(3 ** i)
        raise Exception("Failed to send prompt")
    return wrapper


In [11]:
def produce_example(input, output):
    instruction = """
        You are given the following Input and Output, use 5 words or less to capture the MAIN IDEAS of the output.
        If you were to provide someone with the input and expect them to produce the same output, what 5 words would you use to describe the output?
        Provide comma separated words.

        ### Input
        {input}

        ### Output
        {output}

        USE GENERAL OVERARCHING CONCEPTS AND IDEAS.
        
    """.format(input=input, output=output)
    return send_prompt(instruction)


In [12]:
produce_example = retry_wrapper(produce_example)

In [7]:
from datasets import load_dataset

/Users/aw/projects/phineas/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
openhermes = load_dataset("teknium/openhermes", streaming=True)

In [15]:
import json

openhermes = openhermes['train'].shuffle()

with open('openhermes-concepts.json', 'a') as f:
    for example in openhermes:
        input = example['instruction']
        if example["input"] is not None:
            input += example['input']
        output = example['output']

        try:
            generated = produce_example(input, output)
        except Exception as e:
            print(e)
            continue

        json.dump({
            "instruction": input,
            "output": output,
            "input": generated
        }, f)
        f.write('\n')
        f.flush()

Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
524 Server Error:  for url: https://api.together.xyz/api/inference
Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
524 Server Error:  for url: https://api.together.xyz/api/inference
524 Server Error:  for url: https://api.together.xyz/api/inference
Too many requests received. Please pace your requests.
524 Server Error:  for url: https://api.together.xyz/api/inference
Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
524 Server Error:  for url: https://api.together.xyz/api/inference
Too many requests received. Please pace your requests.
Too many requests received. Please pace your requests.
Too m

KeyboardInterrupt: 